In [6]:
# 2000 tmin is not complete yet. So nyear < 8

# Total size: 365*24*1405*621*5 = 38 billions. There are 4 bytes in one real, so it's 152 GB.
# I need to divide it at least by 100 for memory purpose. I can do that by averaging on regions.

nyear = 1 # Up to 24
nleapday = 1 # Up to 6
nx, ny = 140, 62 # Will give steps of 10 or 11
#nx, ny = 88, 39 # Will give steps of 16 or 17
#nx, ny = 351, 155 # Will give steps of 4 or 5.

# The biggest fire is 600,000 acres = 2428 km2 = 49 km*49 km. By default, the area are 4 km*4 km.
# I can combine region afterwards, so doing large region is useful only to reduce data.

import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
import rasterio
import sqlite3
import pandas as pd

# Split data into region of latitude and longitude.
ybin = np.linspace(0,621,ny, dtype=int)
xbin = np.linspace(0,1405,nx, dtype=int)
ds = rasterio.open('PRISMdaily\PRISM_ppt_stable_4kmD2_19920101_bil.bil')
affine = ds.affine
data = ds.read(1)
latbin = ybin*affine[4]+affine[5]
longbin = xbin*affine[0]+affine[2]
loc = []
for ix in range(nx-1):
    for iy in range(ny-1):
        # Verify we have a full cell.
        inputData0 = data[ybin[iy]:ybin[iy+1],xbin[ix]:xbin[ix+1]]
        if np.any(inputData0==-9999.):
            continue
        loc.append((ix, iy))
loc = np.array(loc)

# day
nday0 = [31,28,31,30,31,30,31,31,30,31,30,31]
nday1 = [31,29,31,30,31,30,31,31,30,31,30,31]
def nday(year,month):
    if year%4 == 0:
        return nday1[month-1]+1
    else:
        return nday0[month-1]+1

# Prepare input data
inputData = []
for year in range(1992,1992+nyear):
  for month in range(1,13):
    for day in range(1,nday(year,month)):
        ds1 = rasterio.open('PRISMdaily\PRISM_ppt_stable_4kmD2_'+str(year)+str(month).zfill(2)+str(day).zfill(2)+'_bil.bil')
        ds2 = rasterio.open('PRISMdaily\PRISM_tmax_stable_4kmD1_'+str(year)+str(month).zfill(2)+str(day).zfill(2)+'_bil.bil')
        ds3 = rasterio.open('PRISMdaily\PRISM_tmean_stable_4kmD1_'+str(year)+str(month).zfill(2)+str(day).zfill(2)+'_bil.bil')
        ds4 = rasterio.open('PRISMdaily\PRISM_tmin_stable_4kmD1_'+str(year)+str(month).zfill(2)+str(day).zfill(2)+'_bil.bil')
        data = np.array([ds1.read(1),ds2.read(1),ds3.read(1),ds4.read(1)])
        inputDataT=[]
        for ix, iy in loc:
            inputDataT.append(np.mean(data[:,ybin[iy]:ybin[iy+1],xbin[ix]:xbin[ix+1]],axis=(1,2)))
        inputData.append(inputDataT)
inputData = np.array(inputData)

# Prepare target
conn = sqlite3.connect('FPA_FOD_20170508.sqlite')
c = conn.cursor()
c.execute('''SELECT DISCOVERY_DATE, FIRE_SIZE, LATITUDE, LONGITUDE 
FROM FIRES''')
rows = c.fetchall()
df = pd.DataFrame(rows)
df.columns = ['time', 'size', 'lat', 'lon']

# drop Alaska, Hawaii, Puerto Rico
# Or maybe use BoundingBox(left=-125.02083333333333, bottom=24.06249999999996, right=-66.47916666666197, top=49.93750000000203)
df = df[df['lon']>-130.]
df = df[df['lat']>24.]
df = df[df['lat']<50.]

target = []
for ix, iy in loc:
    tmp = df[ (df['lon']>longbin[ix]) &  (df['lon']<longbin[ix+1]) &  (df['lat']<latbin[iy]) &  
             (df['lat']>latbin[iy+1]) ]
    dataT = []
    for it in range(2448623,2448623+365*nyear+nleapday):
        tmp2 = tmp[(tmp['time']>it) & (tmp['time']<it+1)]
        if tmp2.empty:
            dataT.append(0)
            continue
        dataT.append(np.sum(tmp2['size'].values))
    target.append(np.array(dataT))
target = np.array(target)

In [7]:
# I made a mistake and missed february 9 for inputData

print(target.shape)
print(loc.shape)
print(inputData.shape)

(4389, 366)
(4389, 2)
(365, 4389, 4)


In [8]:
# drop february 29 for target (I won't need that later)

# february 29 is the 29+31 rows, it starts at 0, so it's 29+31-1=59
# axis=1 since it's the time
target = np.delete(target, (59), axis=1)

print(target.shape)

(4389, 365)


In [11]:
# Flip axis for inputData
inputData = np.swapaxes(inputData,0,1)
print(inputData.shape)
print(target.shape)

# target is (number of xy localization, number of days)
# inputData is (number of xy localization, number of days, 4)
# loc is (number of xy localization,2)

# I need to combine target and inputData0 in one file and save it.
# I need to save loc since it can be useful to combine data

Data = np.concatenate((inputData, target[:,:,np.newaxis]),axis=2)
# It will be (number of xy localization, number of days, 5)

np.save('Data', Data, allow_pickle=True, fix_imports=False)
# !!!Should test it on other notebook first!!!

np.save('LocalizationData', loc, allow_pickle=True, fix_imports=False)

# For 365 days, 4389 xy bins, it's 62 MB
# For 24 years, 4389 xy bins, it will be 1.5 GB

(4389, 365, 4)
(4389, 365)


I now need to use this data for forecasting.

I could use RNN or VARIMA (or ARIMAX), though VARIMA is used when one to also use the previous state of the forecasted time series, when I only want to use the weather time series to forecast the fire one. Someone else talked about Kalman filters or state-space models (SSMs).

I don't think I should remove the seasonality and the trend since it should be determined from the weather data. Some special day like July 4th can affect my results, it might be worth it to correct the fire excedent due to July 4th.